# Ariel spectroscopy ranking — combined plots + eclipse score saturation diagnostics

This notebook loads your exported ranked lists:

- `tpc_transit_ranked.csv`
- `tpc_eclipse_ranked.csv`

and reproduces / extends the combined visualizations **with Transit vs Eclipse overlaid**, then diagnoses why the **eclipse scores saturate near 1** and offers two fixes:

1) **Post-hoc tie-breaker scoring** (no retraining; fastest)
2) **Cleaner retrain**: exclude availability from features + use a **count likelihood** (Negative Binomial) so the model can produce a wider range of required events.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ---------- paths ----------
transit_csv = Path("tpc_transit_ranked.csv")
eclipse_csv = Path("tpc_eclipse_ranked.csv")

assert transit_csv.exists(), f"Missing {transit_csv.resolve()}"
assert eclipse_csv.exists(), f"Missing {eclipse_csv.resolve()}"

tr = pd.read_csv(transit_csv)
ec = pd.read_csv(eclipse_csv)

# IMPORTANT: guard against hidden whitespace in headers
tr.columns = tr.columns.str.strip()
ec.columns = ec.columns.str.strip()

tr.head(), ec.head()


In [ ]:
# ---------- column names used throughout ----------
TR_SCORE = "transit_spectroscopy_score"
EC_SCORE = "eclipse_spectroscopy_score"

TR_PRED  = "pred_Tier1Transits_med"
EC_PRED  = "pred_Tier1Eclipses_med"

TR_PFEAS = "p_pred_le_available_transits"
EC_PFEAS = "p_pred_le_available_eclipses"

TR_AVAIL = "Available Transits"
EC_AVAIL = "Available Eclipses"

needed = [
    (tr, [TR_SCORE, TR_PRED, TR_PFEAS, TR_AVAIL]),
    (ec, [EC_SCORE, EC_PRED, EC_PFEAS, EC_AVAIL]),
]
for df, cols in needed:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        print("Missing columns:", missing)
        print("Available score-like columns:", [c for c in df.columns if "score" in c.lower()])
        raise KeyError(missing)

# ensure sorted best→worst
tr = tr.sort_values(TR_SCORE, ascending=False).reset_index(drop=True)
ec = ec.sort_values(EC_SCORE, ascending=False).reset_index(drop=True)

print("Transit N =", len(tr), "Eclipse N =", len(ec))


## 1) Quick saturation check (why eclipse scores look “overfit”)

If your score is basically:

\[
\text{score} \approx \frac{\Pr(\text{required} \le \text{available})}{\text{required (median)}}
\]

then it will **saturate near 1** when:

- predicted required events are often **≈ 1**, and
- feasibility \(\Pr(\cdot)\) is often **≈ 1** (because available events are large)

That’s not “overfitting” in the usual sense — it’s **metric saturation / degeneracy**: many rows get essentially the same best-possible score.


In [ ]:
def _as_float(df, col):
    return pd.to_numeric(df[col], errors="coerce").to_numpy(float)

def summarize(df, score_col, pred_col, pfeas_col, avail_col):
    s = _as_float(df, score_col)
    pred = _as_float(df, pred_col)
    pfeas = _as_float(df, pfeas_col)
    avail = _as_float(df, avail_col)
    return {
        "N": len(df),
        "score_min": np.nanmin(s),
        "score_med": np.nanmedian(s),
        "score_max": np.nanmax(s),
        "frac_score_eq_1": np.nanmean(np.isclose(s, 1.0)),
        "pred_med_min": np.nanmin(pred),
        "pred_med_med": np.nanmedian(pred),
        "pred_med_max": np.nanmax(pred),
        "frac_pred_med_eq_1": np.nanmean(np.isclose(pred, 1.0)),
        "pfeas_med": np.nanmedian(pfeas),
        "frac_pfeas_eq_1": np.nanmean(np.isclose(pfeas, 1.0)),
        "avail_med": np.nanmedian(avail),
        "avail_min": np.nanmin(avail),
        "avail_max": np.nanmax(avail),
    }

pd.DataFrame({
    "Transit": summarize(tr, TR_SCORE, TR_PRED, TR_PFEAS, TR_AVAIL),
    "Eclipse": summarize(ec, EC_SCORE, EC_PRED, EC_PFEAS, EC_AVAIL),
}).T


In [ ]:
# Value counts: do we only predict a few discrete required-event levels?
print("Transit predicted required (median) value counts:")
display(pd.to_numeric(tr[TR_PRED], errors="coerce").value_counts().sort_index())

print("\nEclipse predicted required (median) value counts:")
display(pd.to_numeric(ec[EC_PRED], errors="coerce").value_counts().sort_index())


In [ ]:
# ---------- diagnostic plots: required / feasibility / availability ----------
C_TR = "tab:blue"
C_EC = "tab:orange"

tr_pred  = _as_float(tr, TR_PRED)
ec_pred  = _as_float(ec, EC_PRED)
tr_pfeas = _as_float(tr, TR_PFEAS)
ec_pfeas = _as_float(ec, EC_PFEAS)
tr_av   = _as_float(tr, TR_AVAIL)
ec_av   = _as_float(ec, EC_AVAIL)

plt.figure(figsize=(7.5, 4.8))
plt.hist(tr_pred[np.isfinite(tr_pred)], bins=np.arange(0.5, np.nanmax(tr_pred)+1.5, 1), alpha=0.6, label="Transit", color=C_TR)
plt.hist(ec_pred[np.isfinite(ec_pred)], bins=np.arange(0.5, np.nanmax(ec_pred)+1.5, 1), alpha=0.6, label="Eclipse", color=C_EC)
plt.xlabel("Predicted required Tier-1 events (median)")
plt.ylabel("count")
plt.title("Predicted required events distribution")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7.5, 4.8))
plt.hist(tr_pfeas[np.isfinite(tr_pfeas)], bins=30, alpha=0.6, label="Transit", color=C_TR)
plt.hist(ec_pfeas[np.isfinite(ec_pfeas)], bins=30, alpha=0.6, label="Eclipse", color=C_EC)
plt.xlabel("P(required ≤ available)")
plt.ylabel("count")
plt.title("Feasibility distribution")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7.5, 4.8))
plt.hist(np.log10(tr_av[np.isfinite(tr_av)]), bins=40, alpha=0.6, label="Transit", color=C_TR)
plt.hist(np.log10(ec_av[np.isfinite(ec_av)]), bins=40, alpha=0.6, label="Eclipse", color=C_EC)
plt.xlabel("log10(Available events)")
plt.ylabel("count")
plt.title("Availability distribution (log10 scale)")
plt.legend()
plt.tight_layout()
plt.show()


## 2) Reproduce the combined plots you showed (overlay Transit + Eclipse)

These match your combined figures:
- score drop-off vs rank
- feasibility vs required (Pareto-ish view)
- depth vs brightness (marker size ∝ score)
- score distributions (overlaid histogram)


In [ ]:
def transit_depth_frac(df):
    d = pd.to_numeric(df.get("Transit Depth [%]"), errors="coerce").to_numpy(float)
    return d / 100.0

def tess_mag(df):
    return pd.to_numeric(df.get("Star TESS Mag"), errors="coerce").to_numpy(float)

def size_from_score(s, smin=None, smax=None):
    s = np.asarray(s, float)
    if smin is None: smin = np.nanmin(s)
    if smax is None: smax = np.nanmax(s)
    return 20 + 240 * (s - smin) / (smax - smin + 1e-12)

tr_score = _as_float(tr, TR_SCORE)
ec_score = _as_float(ec, EC_SCORE)

# ---------- Score drop-off ----------
plt.figure(figsize=(7.5, 4.8))
plt.plot(np.arange(1, len(tr)+1), tr_score, label="Transit", color=C_TR)
plt.plot(np.arange(1, len(ec)+1), ec_score, label="Eclipse", color=C_EC)
plt.xlabel("Rank (1 = best)")
plt.ylabel("spectroscopy score")
plt.title("Score drop-off across ranked lists (Transit vs Eclipse)")
plt.legend()
plt.tight_layout()
plt.show()

# ---------- Feasibility vs required ----------
topN = 80
plt.figure(figsize=(7.2, 6.2))
plt.scatter(tr_pred, tr_pfeas, alpha=0.18, label="Transit", color=C_TR)
plt.scatter(ec_pred, ec_pfeas, alpha=0.18, label="Eclipse", color=C_EC)
plt.scatter(tr_pred[:topN], tr_pfeas[:topN], alpha=0.95, color=C_TR)
plt.scatter(ec_pred[:topN], ec_pfeas[:topN], alpha=0.95, color=C_EC)
plt.xlabel("Predicted required Tier-1 events (median)")
plt.ylabel("P(required ≤ available)")
plt.title(f"Feasibility vs required events (top {topN} highlighted)")
plt.legend()
plt.tight_layout()
plt.show()

# ---------- Depth vs brightness (marker size ∝ score) ----------
tr_depth = transit_depth_frac(tr)
ec_depth = transit_depth_frac(ec)
tr_mag = tess_mag(tr)
ec_mag = tess_mag(ec)

all_scores = np.concatenate([tr_score[np.isfinite(tr_score)], ec_score[np.isfinite(ec_score)]])
smin, smax = np.nanmin(all_scores), np.nanmax(all_scores)
tr_ms = size_from_score(tr_score, smin=smin, smax=smax)
ec_ms = size_from_score(ec_score, smin=smin, smax=smax)

plt.figure(figsize=(7.8, 5.6))
plt.scatter(tr_depth, tr_mag, s=tr_ms, alpha=0.20, label="Transit", color=C_TR)
plt.scatter(ec_depth, ec_mag, s=ec_ms, alpha=0.20, label="Eclipse", color=C_EC)
plt.xscale("log")
plt.gca().invert_yaxis()
plt.xlabel("Transit Depth (fraction)")
plt.ylabel("Star TESS Mag (brighter = higher)")
plt.title("Depth vs brightness (marker size ∝ score)")
plt.legend()
plt.tight_layout()
plt.show()

# ---------- Score distributions ----------
plt.figure(figsize=(7.5, 4.8))
plt.hist(tr_score[np.isfinite(tr_score)], bins=40, alpha=0.55, label="Transit", color=C_TR)
plt.hist(ec_score[np.isfinite(ec_score)], bins=40, alpha=0.55, label="Eclipse", color=C_EC)
plt.xlabel("spectroscopy score")
plt.ylabel("count")
plt.title("Score distributions (Transit vs Eclipse)")
plt.legend()
plt.tight_layout()
plt.show()


## 3) Fix #1 (fast): post-hoc **tie-breaker** score for eclipse

When many eclipse rows have **score ≈ 1**, your list is dominated by ties.  
A simple fix (without retraining) is:

- keep the feasibility/required score as the **base**
- break ties using **brightness** (IR/TESS mags), **planet temperature** (for eclipse), and **depth SNR** (for transit)

This keeps your intended meaning (“feasibility / required”) while still ranking targets sensibly *within the saturated region*.


In [ ]:
def percentile_rank(x):
    x = np.asarray(x, float)
    out = np.zeros_like(x, float)
    ok = np.isfinite(x)
    if ok.sum() < 2:
        return out
    r = x[ok].argsort().argsort().astype(float)
    out[ok] = r / (len(r) - 1)
    return out

def mag_to_flux(m):
    m = np.asarray(m, float)
    out = np.full_like(m, np.nan, float)
    ok = np.isfinite(m)
    out[ok] = 10.0 ** (-0.4 * m[ok])
    return out

def ir_flux_proxy(df):
    # prefer IR mags if present; otherwise fallback to TESS
    mags = []
    for c in ["Star J Mag", "Star H Mag", "Star Ks Mag", "Star W1 Mag", "Star W2 Mag"]:
        if c in df.columns:
            mags.append(mag_to_flux(pd.to_numeric(df[c], errors="coerce").to_numpy(float)))
    if len(mags) > 0:
        return np.nanmean(np.vstack(mags), axis=0)
    return mag_to_flux(tess_mag(df))

def depth_snr(df):
    depth = pd.to_numeric(df.get("Transit Depth [%]"), errors="coerce").to_numpy(float)
    dlo = pd.to_numeric(df.get("Transit Depth Error Lower [%]"), errors="coerce").to_numpy(float)
    dhi = pd.to_numeric(df.get("Transit Depth Error Upper [%]"), errors="coerce").to_numpy(float)
    sig = 0.5 * (np.abs(dlo) + np.abs(dhi))
    return (depth / np.maximum(sig, 1e-12))

# --- build tie-breaker terms ---
tr_base = tr_score.copy()
ec_base = ec_score.copy()

tr_bright = percentile_rank(ir_flux_proxy(tr))   # brighter -> higher
ec_bright = percentile_rank(ir_flux_proxy(ec))

tr_depthsnr = percentile_rank(depth_snr(tr))
ec_teq = pd.to_numeric(ec.get("Planet Temperature [K]"), errors="coerce").to_numpy(float)
ec_teq_rank = percentile_rank(ec_teq)            # hotter -> higher (for eclipse)

# refined scores:
# Transit: break ties with brightness + depth SNR
tr_refined = tr_base * (0.6 + 0.2*tr_bright + 0.2*tr_depthsnr)

# Eclipse: break ties with brightness + Teq
ec_refined = ec_base * (0.6 + 0.2*ec_bright + 0.2*ec_teq_rank)

# show whether eclipse saturation is reduced
plt.figure(figsize=(7.5, 4.8))
plt.hist(ec_base[np.isfinite(ec_base)], bins=40, alpha=0.55, label="Eclipse base", color="tab:orange")
plt.hist(ec_refined[np.isfinite(ec_refined)], bins=40, alpha=0.55, label="Eclipse refined", color="tab:red")
plt.xlabel("score")
plt.ylabel("count")
plt.title("Eclipse score saturation: base vs refined (tie-breaker)")
plt.legend()
plt.tight_layout()
plt.show()

# write new ranked files
tr_out = tr.copy()
ec_out = ec.copy()
tr_out["transit_score_refined"] = tr_refined
ec_out["eclipse_score_refined"] = ec_refined

tr_out = tr_out.sort_values("transit_score_refined", ascending=False).reset_index(drop=True)
ec_out = ec_out.sort_values("eclipse_score_refined", ascending=False).reset_index(drop=True)

tr_out.to_csv("tpc_transit_ranked_refined.csv", index=False)
ec_out.to_csv("tpc_eclipse_ranked_refined.csv", index=False)

print("Wrote:")
print(" - tpc_transit_ranked_refined.csv")
print(" - tpc_eclipse_ranked_refined.csv")

# quick peek
cols_show_tr = ["Planet Name","Star Name",TR_SCORE,"transit_score_refined",TR_PRED,TR_PFEAS,TR_AVAIL,"Star TESS Mag","Planet Temperature [K]"]
cols_show_ec = ["Planet Name","Star Name",EC_SCORE,"eclipse_score_refined",EC_PRED,EC_PFEAS,EC_AVAIL,"Star TESS Mag","Planet Temperature [K]"]
cols_show_tr = [c for c in cols_show_tr if c in tr_out.columns]
cols_show_ec = [c for c in cols_show_ec if c in ec_out.columns]

display(tr_out[cols_show_tr].head(15))
display(ec_out[cols_show_ec].head(15))


## 4) Fix #2 (cleaner): remove **availability** from the model features (avoid leakage)

If your feature set includes `Available Transits` / `Available Eclipses`, the model can partially learn schedule/availability structure,
which then makes predictions collapse toward **“required ≈ 1”** whenever availability is large.

Cleaner setup:

- **Model predicts required events** from: depth + depth errors + duration errors + stellar magnitudes + Teq  
  (exclude `Available ...`)
- **Scoring uses availability only in feasibility**: \(\Pr(\text{required} \le \text{available})\)

Below is a drop-in sketch of a better likelihood for counts, too.


In [ ]:
# ---- COUNT likelihood idea (recommended for targets like 'Tier 1 Eclipses') ----
# Instead of modeling log(count) as a continuous variable, model counts directly with a log link.
# This produces integer predictive draws and typically a wider range of required-event predictions.

import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist

def negbin_count_model(x, y=None):
    """Negative Binomial regression: y ~ NB(mean=exp(b0 + x beta), dispersion=phi)."""
    n, p = x.shape
    b0 = numpyro.sample("b0", dist.Normal(0, 1))
    beta = numpyro.sample("beta", dist.Normal(0, 1).expand((p,)))
    # dispersion / overdispersion (larger phi -> closer to Poisson)
    phi = numpyro.sample("phi", dist.HalfNormal(1.0))

    eta = b0 + jnp.dot(x, beta)
    mu = jnp.exp(eta)

    # NB parameterization in NumPyro: NegativeBinomial2(mean=mu, concentration=phi)
    numpyro.sample("y", dist.NegativeBinomial2(mean=mu, concentration=phi), obs=y)

# Notes:
# - You can keep your existing StandardScaler + imputer.
# - SVI works well for this too.
# - Scoring: use posterior predictive draws y_draws (counts) directly.


### Minimal scoring change if you switch to a count likelihood

If `y_draws` are integer required-event draws:

- `pred_required_med = median(y_draws)`
- `p_feasible = mean(y_draws <= available)`
- `score = p_feasible / pred_required_med`

This usually avoids the “everything ≈ 1” collapse if the model can predict a broader count range.

---

### If you want, I can convert your current `ariel_ranking_no_waic_with_mags_v2.py` into a notebook
and implement the Negative Binomial option behind a CLI flag like:

- `--likelihood lognormal` (current)
- `--likelihood negbin` (recommended for counts)
